<a href="https://colab.research.google.com/github/deep-l3arn3r/tutorials/blob/main/torch_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Download the dataset***

In [1]:
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

data/pizza_steak_sushi directory exists.


In [2]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Create simple transform
simple_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [5]:
# 1. Load and transform data
from torchvision import datasets
train_data_simple = datasets.ImageFolder(root=train_dir, transform=simple_transform)
test_data_simple = datasets.ImageFolder(root=test_dir, transform=simple_transform)

# 2. Turn data into DataLoaders
import os
from torch.utils.data import DataLoader

# Setup batch size and number of workers
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS)

train_dataloader_simple, test_dataloader_simple

Creating DataLoader's with batch size 32 and 2 workers.


(<torch.utils.data.dataloader.DataLoader at 0x7c35159856f0>,
 <torch.utils.data.dataloader.DataLoader at 0x7c3515985d80>)

In [8]:
import torch.nn as nn

class TinyVGGModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):

    super().__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=input_dim,
            out_channels=hidden_dim,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.block3 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=16*16*hidden_dim, out_features=output_dim)
    )


  def forward(self, x):
    x = self.conv1(x)

    x = self.conv2(x)

    x = self.block3(x)

    return x

In [60]:
torch.manual_seed(42)
model_0 = TinyVGGModel(input_dim=3, # number of color channels (3 for RGB)
                  hidden_dim=5,
                  output_dim=len(train_data_simple.classes))
model_0

TinyVGGModel(
  (conv1): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1280, out_features=3, bias=True)
  )
)

In [61]:
def train_step(model, dataloader, loss_fn, optimizer):

  model.train()

  train_loss, train_accuracy = 0, 0

  for batch, (X, y) in enumerate(dataloader):

    y_pred_logit = model(X)

    loss = loss_fn(y_pred_logit, y)
    train_loss += loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    y_pred_class = torch.argmax(y_pred_logit, dim=1)
    train_accuracy += (y_pred_class==y).sum().item()/len(y)

  train_loss = train_loss/len(dataloader)
  train_accuracy = train_accuracy/len(dataloader)

  return train_loss, train_accuracy


def test_step(model, dataloader, loss_fn):

  model.eval()

  test_loss, test_accuracy = 0, 0

  with torch.inference_mode():

    for batch, (X, y) in enumerate(dataloader):

      y_pred_logit = model(X)

      loss = loss_fn(y_pred_logit, y)
      test_loss += loss.item()

      y_pred_class = torch.argmax(y_pred_logit, dim=1)
      test_accuracy += (y_pred_class==y).sum().item()/len(y)

  test_loss = test_loss/len(dataloader)
  test_accuracy = test_accuracy/len(dataloader)

  return test_loss, test_accuracy

In [62]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [63]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

In [64]:
results = train(model_0, train_dataloader_simple, test_dataloader_simple, optimizer, loss_fn, 50)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1013 | train_acc: 0.2227 | test_loss: 1.0985 | test_acc: 0.2604
Epoch: 2 | train_loss: 1.0844 | train_acc: 0.4258 | test_loss: 1.1274 | test_acc: 0.2604
Epoch: 3 | train_loss: 1.0796 | train_acc: 0.4258 | test_loss: 1.1537 | test_acc: 0.2604
Epoch: 4 | train_loss: 1.1276 | train_acc: 0.3047 | test_loss: 1.1585 | test_acc: 0.2604
Epoch: 5 | train_loss: 1.1234 | train_acc: 0.3047 | test_loss: 1.1363 | test_acc: 0.2604
Epoch: 6 | train_loss: 1.0964 | train_acc: 0.4258 | test_loss: 1.1088 | test_acc: 0.2604
Epoch: 7 | train_loss: 1.1005 | train_acc: 0.3047 | test_loss: 1.1061 | test_acc: 0.2604
Epoch: 8 | train_loss: 1.1043 | train_acc: 0.3242 | test_loss: 1.1109 | test_acc: 0.1771
Epoch: 9 | train_loss: 1.0991 | train_acc: 0.3359 | test_loss: 1.1011 | test_acc: 0.1979
Epoch: 10 | train_loss: 1.0996 | train_acc: 0.2734 | test_loss: 1.0912 | test_acc: 0.5417
Epoch: 11 | train_loss: 1.0953 | train_acc: 0.4023 | test_loss: 1.0911 | test_acc: 0.5417
Epoch: 12 | train_l